# Used Cars Price Prediction

### Loading Libraries

In [1]:
# !python chapter_setup.py

In [2]:
# Numerical Computing
import math
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt 

# Operating System
import os, random

# Dataset Source
import machine_learning_datasets as mldatasets

# Scikit-Learn
from sklearn import metrics, ensemble, tree, inspection, model_selection

# CatBoost
import catboost as cb

# Model Interpreation
import shap
from PyALE import ale
from pdpbox import pdp, info_plots
from lime.lime_tabular import LimeTabularExplainer

### Understanding & Preparing Data

In [3]:
usedcars_df = mldatasets.load("usedcars", prepare=True)

In [4]:
usedcars_orig_df = usedcars_df.copy()
usedcars_df = mldatasets.make_dummies_with_limits(usedcars_df, 'fuel',\
                                                  min_recs=500, defcatname='other')
usedcars_df = mldatasets.make_dummies_with_limits(usedcars_df, 'make_cat')
usedcars_df = mldatasets.make_dummies_with_limits(usedcars_df, 'model_type',\
                                                  min_recs=500, defcatname='other')
usedcars_df = mldatasets.make_dummies_with_limits(usedcars_df, 'condition',\
                                                  min_recs=200, defcatname='other')
usedcars_df = mldatasets.make_dummies_with_limits(usedcars_df, 'drive')
usedcars_df = mldatasets.make_dummies_with_limits(usedcars_df, 'title_status',\
                                                  min_recs=500, defcatname='other')

In [5]:
usedcars_df.dtypes[lambda x: x == object].index.tolist()

In [6]:
usedcars_df = usedcars_df.select_dtypes(include=(int,float,np.uint8))

In [7]:
# Randomness
rand = 42
os.environ['PYTHONHASHSEED']=str(rand)
np.random.seed(rand)
random.seed(rand)

# Target
target_col = 'price'

# Features
X = usedcars_df.drop([target_col], axis=1)
y = usedcars_df[target_col]

# Train-Test Split
X_train, X_test, y_train, y_test =\
        model_selection.train_test_split(X, y, test_size=0.25, random_state=rand)

### Model Training & Evaluation

In [ ]:
# Cat Model Set
cb_mdl = cb.CatBoostRegressor(depth=7, learning_rate=0.2,\
                               random_state=rand, verbose=False)

# Cat Fitting Model
cb_mdl = cb_mdl.fit(X_train, y_train)

# RandomForest ModelSet
rf_mdl = ensemble.RandomForestRegressor(n_jobs=-1, random_state=rand)

# RandomForest Fitting Model
rf_mdl = rf_mdl.fit(X_train.to_numpy(), y_train.to_numpy())

In [8]:
mdl = cb_mdl

sns.set()
y_train_pred, y_test_pred =\
        mldatasets.evaluate_reg_mdl(mdl, X_train, X_test, y_train, y_test)

In [9]:
thresh = 4000

pct_under = np.where(np.abs(y_test - y_test_pred) < thresh, 1, 0).sum() / len(y_test)
print(f"Percentage of test samples under ${thresh:,.0f} in absolute error {pct_under:.1%}")

In [10]:
mdl = rf_mdl

sns.set()
_, _ =\
        mldatasets.evaluate_reg_mdl(mdl, X_train, X_test, y_train, y_test)

### What is Feature Importance?

In [11]:
fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (16,8), dpi=600)
tree.plot_tree(rf_mdl.estimators_[0], feature_names=X_train.columns,\
               filled = True, max_depth=2, fontsize=14)
fig.show()

In [12]:
rf_feat_imp = rf_mdl.feature_importances_
print(rf_feat_imp)

In [13]:
sum(rf_feat_imp)

In [14]:
cb_feat_imp = cb_mdl.feature_importances_
print(cb_feat_imp)

In [15]:
sum(cb_feat_imp)

In [16]:
feat_imp_df = pd.DataFrame({'feature':X_train.columns, 'cb_feat_imp':cb_feat_imp,\
                            'rf_feat_imp':rf_feat_imp*100})

feat_imp_df = feat_imp_df.sort_values('cb_feat_imp', ascending=False)
feat_imp_df.style.format('{:.2f}%', subset=['cb_feat_imp', 'rf_feat_imp']).\
                bar(subset=['cb_feat_imp', 'rf_feat_imp'], color='#4EF', width=60)

### Assessing Feature Importance with Model-agnostic Methods

#### Permutation Feature Importance

In [17]:
%%time
X_samp = X_test.sample(frac=0.3)
y_samp = y_test.loc[X_samp.index]

cb_perm_imp = inspection.permutation_importance(cb_mdl, X_samp, y_samp,\
                                                n_repeats=5, random_state=rand,\
                                                scoring='neg_mean_absolute_error')
rf_perm_imp = inspection.permutation_importance(rf_mdl, X_samp.to_numpy(),\
                                                y_samp.to_numpy(), n_repeats=5,\
                                                random_state=rand, scoring='neg_mean_absolute_error')

In [18]:
perm_imp_df = pd.DataFrame({'feature':X_train.columns, 'cb_perm_mean':cb_perm_imp.importances_mean,\
                            'cb_perm_std':cb_perm_imp.importances_std,\
                            'rf_perm_mean':rf_perm_imp.importances_mean,\
                            'rf_perm_std':rf_perm_imp.importances_std})

perm_imp_df = perm_imp_df.sort_values('cb_perm_mean', ascending=False)
perm_imp_df.style.format('{:.4f}', subset=['cb_perm_mean', 'cb_perm_std', 'rf_perm_mean', 'rf_perm_std']).\
                bar(subset=['cb_perm_mean', 'rf_perm_mean'], color='#4EF', width=60)

## SHAP Values

### Comprehensive Explanations with the KernelExplainer

In [19]:
%%time
rf_fn = lambda x: rf_mdl.predict(x)
X_train_summary = shap.kmeans(X_train.to_numpy(), 50)
X_test_sample = X_test.sample(frac=0.02)

rf_kernel_explainer = shap.KernelExplainer(rf_fn, X_train_summary)
rf_shap_values = rf_kernel_explainer.shap_values(X_test_sample.to_numpy(),\
                                                 nsamples=100) 

#### Faster Explanations with TreeExplainer

In [20]:
%%time
cb_tree_explainer = shap.TreeExplainer(cb_mdl)
cb_shap_values = cb_tree_explainer.shap_values(X_test)

In [21]:
cb_shap_imp = np.mean(np.abs(cb_shap_values),0)

In [22]:
rf_shap_imp = np.mean(np.abs(rf_shap_values),0)

In [23]:
shap_imp_df = pd.DataFrame({'feature':X_train.columns, 'cb_shap_imp':cb_shap_imp,\
                            'rf_shap_imp':rf_shap_imp})

shap_imp_df = shap_imp_df.sort_values('cb_shap_imp', ascending=False)
shap_imp_df.style.format('{:.4f}', subset=['cb_shap_imp', 'rf_shap_imp']).\
                bar(subset=['cb_shap_imp', 'rf_shap_imp'], color='#4EF', width=60)

### Visualize Global Explanations

In [24]:
%%time
cb_explainer = shap.Explainer(cb_mdl)
cb_shap = cb_explainer(X_test)

In [25]:
print(type(cb_explainer))

In [26]:
print("Values dimensions: %s" % (cb_shap.values.shape,)) 
print("Data dimensions:   %s" % (cb_shap.data.shape,))

#### SHAP Bar Plot

In [27]:
sns.reset_orig()

shap.plots.bar(cb_shap, max_display=15) 

In [28]:
yr_cohort = np.where(X_test.year > 2014,\
                     "Newer Car", "Older Car")

shap.plots.bar(cb_shap.cohorts(yr_cohort).abs.mean(0))

#### SHAP Beeswarm Plot

In [29]:
shap.plots.beeswarm(cb_shap, max_display=15)

### Feature summary explanations

#### Partial Dependence Plot (PDP)

In [ ]:
shap.plots.partial_dependence("year", cb_mdl.predict, X_test,\
                              ice=False, model_expected_value=True,\
                              feature_expected_value=True)

In [30]:
pdp_single_feature = pdp.PDPIsolate(
    model=cb_mdl, df=X_test, model_features=X_test.columns, feature='year',\
    feature_name='year', n_classes=0, n_jobs=-1
)
fig, axes = pdp_single_feature.plot(plot_pts_dist=True)
fig.show()

In [31]:
fig, axes = pdp_single_feature.plot(plot_pts_dist=True, plot_lines=True,\
                                    frac_to_plot=0.01)
fig.show()

In [32]:
fig, axes = pdp_single_feature.plot(to_bins=True, plot_lines=True,\
                                    frac_to_plot=0.01, show_percentile=True)
fig.show()

In [33]:
cont_feature_l = ['year', 'odometer', 'cylinders', 'epa_displ',\
                  'make_pop', 'make_yr_pop', 'model_yr_pop']

make_cat_feature_l = ['make_cat_obsolete', 'make_cat_regular', 'make_cat_premium',\
                      'make_cat_luxury', 'make_cat_luxury_sports']

bin_feature_l = ['model_premier', 'auto_trans']

cat_feature_l = make_cat_feature_l + bin_feature_l

In [34]:
for feature in cont_feature_l: 
    pdp_single_feature = pdp.PDPIsolate(
        model=cb_mdl, df=X_test, model_features=X_test.columns,\
        feature=feature, feature_name=feature, n_classes=0, n_jobs=-1)
    fig, axes = pdp_single_feature.plot(to_bins=True, plot_lines=True,\
                             frac_to_plot=0.01, show_percentile=True,\
                             engine='matplotlib')
    plt.show()

In [35]:
for feature in make_cat_feature_l: 
    pdp_single_feature = pdp.PDPIsolate(
        model=cb_mdl, df=X_test, model_features=X_test.columns,\
        feature=feature, feature_name=feature, n_classes=0, n_jobs=-1)
    fig, axes = pdp_single_feature.plot(to_bins=True, plot_lines=True,\
                             frac_to_plot=0.01, show_percentile=True,\
                             engine='matplotlib')
    plt.show()

In [36]:
pdp_multi_feature = pdp.PDPIsolate(
    model=cb_mdl, df=X_test, model_features=X_test.columns,\
    feature=make_cat_feature_l, feature_name="make_cat",\
    n_classes=0, n_jobs=-1)

fig, axes = pdp_multi_feature.plot(plot_lines=True, plot_pts_dist=True,\
                         frac_to_plot=0.01, show_percentile=True)
fig.show()

In [37]:
predict_plot = info_plots.PredictPlot(
    model=cb_mdl, df=X_test, model_features=X_test.columns, feature=make_cat_feature_l,\
    feature_name="make_cat", n_classes=0)

fig, _, _ = predict_plot.plot()
fig.show()

#### SHAP Scatter Plot

In [38]:
shap.plots.scatter(cb_shap[:,"odometer"], color=cb_shap[:,"year"],\
                   xmin="percentile(1)", xmax="percentile(99)", alpha=0.2)

In [39]:
shap.plots.scatter(cb_shap[:,"long"], color=cb_shap[:,"epa_displ"],\
                   xmin="percentile(0.5)", xmax="percentile(99.5)", alpha=0.2,\
                   x_jitter=0.5)

In [40]:
shap.plots.scatter(cb_shap[:,"make_cat_luxury"], color=cb_shap[:,"year"],\
                   x_jitter=0.4, alpha=0.2, hist=False)

### Accumulated Local Effects (ALE) Plots